<img style="float: center;" src='https://github.com/spacetelescope/jwst-pipeline-notebooks/raw/main/_static/stsci_header.png' alt="stsci_logo" width="900px"/> 

<a id="title_ID"></a>
# MIRI EMICORR Testing Notebook #

**Authors**: Ian Wong; MIRI branch<br>
**Last Updated**: February 25, 2025<br>
**Pipeline Version**: Pull request #9216

**Purpose**:<br>
This notebook compares the performance of the standard (original) algorithm within the `emicorr` step with Timothy Brandt's new methodology.

**Data**:<br>
This example is set up to use the dedicated background observations associated with the LRS slitless observations of the A-type flux calibrator star HR 5467 (PID 4496, Obs 12).<br>

**JWST pipeline version and CRDS context**:<br>
This notebook was written for the calibration pipeline version listed above, which is currently a pull request and not part of a public release.<br>

**Recent Changes**:<br>
Feb 25 2025: Notebook created.

<hr style="border:1px solid gray"> </hr>

## Table of Contents

1. [Configuration](#1.-Configuration)
2. [Package Imports](#2.-Package-Imports)
3. [Demo Mode Setup](#3.-Demo-Mode-Setup-(ignore-if-not-using-demo-data))
4. [Directory Setup](#4.-Directory-Setup)
5. [Detector1 Pipeline](#5.-Detector1-Pipeline)
6. [Compare Outputs](#6.-Compare-Outputs)

<hr style="border:1px solid gray"> </hr>

1.<font color='white'>-</font>Configuration<a class="anchor" id="intro"></a>
------------------
### Install dependencies
To make sure that the pipeline version is compatabile with this notebook and the required dependencies and packages are installed,
it is recommended that users create a new dedicated conda environment and install the provided
`requirements.txt` file before starting this notebook: <br>
```
conda create -n lrs_emicorr_test python=3.12
conda activate lrs_emicorr_test
pip install git+https://github.com/user/jwst.git@refs/pull/9216/head
pip install -r requirements.txt
```

In [ ]:
# Basic import necessary for configuration
import os

<div class="alert alert-block alert-warning">
Note that <code>demo_mode</code> must be set appropriately below.
</div>

Set <code>demo_mode = True </code> to run in demonstration mode. In this mode, this
notebook will download the example data from the
Barbara A. Mikulski Archive for Space Telescopes (MAST) and process them through the pipeline.
All input and output data will be stored in the local directory unless modified
in [Section 3](#3.-Demo-Mode-Setup-(ignore-if-not-using-demo-data)) below. 

Set <code>demo_mode = False</code> to process user-specified data that have already
been downloaded and provide the location of the data.<br>

In [ ]:
# Set parameters for demo_mode, data directory, and processing steps

# -----------------------------Demo Mode---------------------------------
demo_mode = True

if demo_mode:
    print('Running in demonstration mode using online example data!')

# --------------------------User Mode Directories------------------------
# If demo_mode = False, look for user data in these paths
if not demo_mode:
    # Set directory paths for processing specific data; these will need
    # to be changed to the user's local directory setup (paths below are given as
    # examples).
    basedir = os.path.join(os.getcwd(), 'lrs_tso_demo_data')

    # Point to where the observation data are stored.
    # Assumes uncalibrated data in obs_dir/uncal/
    obs_dir = os.path.join(basedir, 'PID04496Obs012/')

### Set CRDS context and server
Before importing <code>CRDS</code> and <code>JWST</code> modules, we need to configure our environment. This includes defining a CRDS cache directory in which to keep the reference files that will be used by the calibration pipeline.

If the root directory for the local CRDS cache directory has not been set already, it will be automatically created in the home directory.

In [ ]:
# ------------------------Set CRDS context and paths----------------------
# Set CRDS reference file context.  Leave commented out to use the default context
# (latest reference files associated with the calibration pipeline version)
# or set a specific context here.
#%env CRDS_CONTEXT  jwst_1295.pmap

# Check whether the local CRDS cache directory has been set.
# If not, set it to the user home directory.
if (os.getenv('CRDS_PATH') is None):
    os.environ['CRDS_PATH'] = os.path.join(os.path.expanduser('~'), 'crds')
    
# Check whether the CRDS server URL has been set.  If not, set it.
if (os.getenv('CRDS_SERVER_URL') is None):
    os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'

# Print out CRDS path and context that will be used
print('CRDS local filepath:', os.environ['CRDS_PATH'])
print('CRDS file server:', os.environ['CRDS_SERVER_URL'])

<hr style="border:1px solid gray"> </hr>

## 2.<font color='white'>-</font>Package Imports<a class="anchor" id="intro"></a>

Automatically import necessary Python packages for use in the data processing and visualization.

In [ ]:
# Use the entire available screen width for this notebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
# Basic system utilities for interacting with files
# ----------------------General Imports------------------------------------
import glob
import time
from pathlib import Path

# Numpy for doing calculations
import numpy as np

# -----------------------Astropy Imports-----------------------------------
# Astropy utilities for opening FITS and ASCII files and downloading demo files
from astropy.io import fits
from astroquery.mast import Observations

# -----------------------Plotting Imports----------------------------------
# Matplotlib for making plots
import matplotlib.pyplot as plt
from matplotlib import rc

In [ ]:
# --------------JWST Calibration Pipeline Imports---------------------------
# Import the base JWST and calibration reference data packages
import jwst
import crds

# JWST pipelines (each encompassing many steps)
from jwst.pipeline import Detector1Pipeline

# JWST pipeline utilities
from jwst import datamodels  # JWST datamodels

# Print out pipeline version and CRDS context that will be used
print("JWST Calibration Pipeline Version = {}".format(jwst.__version__))
print("Using CRDS Context = {}".format(crds.get_context_name('jwst')))

In [ ]:
# Start a timer to keep track of runtime
time0 = time.perf_counter()

<hr style="border:1px solid gray"> </hr>

3.<font color='white'>-</font>Demo Mode Setup<a class="anchor" id="intro"></a> (ignore if not using demo data)
------------------
If running in demonstration mode, set up the program information to
retrieve the uncalibrated data automatically from MAST using
[astroquery](https://astroquery.readthedocs.io/en/latest/mast/mast.html).
MAST allows for flexibility of searching by the proposal ID and the
observation ID instead of just filenames.<br>

More information about the JWST file naming conventions can be found at:
https://jwst-pipeline.readthedocs.io/en/latest/jwst/data_products/file_naming.html

In [ ]:
# Set up the program information and paths for demo program
if demo_mode:
    program = "04496"
    obs_numb = "012"
    basedir = os.path.join('.', 'lrs_demo_data')
    obs_dir = os.path.join(basedir, 'PID' + program + 'Obs' + obs_numb)
    uncal_dir = os.path.join(obs_dir, 'uncal')

    # Ensure filepaths for input data exists
    os.makedirs(uncal_dir, exist_ok=True)

Identify the list of uncalibrated files associated with the science and background observations.

In [ ]:
# Obtain a list of observation IDs for the specified demo program
if demo_mode:
    obs_id_table = Observations.query_criteria(instrument_name=["MIRI/SLITLESS"],
                                                   provenance_name=["CALJWST"],  # Executed observations
                                                   obs_id=['jw' + program + obs_numb + '*']
                                                   )

In [ ]:
# Turn the list of observations into a list of uncalibrated data files
if demo_mode:
    # Define types of files to select
    file_dict = {'uncal': {'product_type': 'SCIENCE', 
                           'productSubGroupDescription': 'UNCAL', 
                           'calib_level': [1]}}

    files_to_download = []
    # Loop over visits identifying uncalibrated files that are associated with them
    for exposure in (obs_id_table):
        products = Observations.get_product_list(exposure)
        for filetype, query_dict in file_dict.items():
            filtered_products = Observations.filter_products(products, productType=query_dict['product_type'],
                                                             productSubGroupDescription=query_dict['productSubGroupDescription'],
                                                             calib_level=query_dict['calib_level'])
            files_to_download.extend(filtered_products['dataURI'])

    print("Number of science files selected for downloading: ", len(files_to_download))

Download all the uncal files and place them into the appropriate directories.

<div class="alert alert-block alert-warning">
Warning: If this notebook is halted during this step, the downloaded files may be incomplete and cause crashes later on!
</div>

In [ ]:
if demo_mode:
    for filename in files_to_download:
        obs_manifest = Observations.download_file(filename, local_path=os.path.join(uncal_dir, Path(filename).name))

<hr style="border:1px solid gray"> </hr>

4.<font color='white'>-</font>Directory Setup<a class="anchor" id="intro"></a>
------------------
Set up detailed paths to input/output stages here. When running this notebook outside of demo mode, the uncalibrated pipeline input files must be placed into the appropriate directories before proceeding to the JWST pipeline processing.

There are separate output directories for the cases using the standard vs. new `emicorr` algorithm.

In [ ]:
# Define output subdirectories to keep science data products organized
uncal_dir = os.path.join(obs_dir, 'uncal')     # Uncalibrated pipeline inputs should be here
det1_dir = os.path.join(obs_dir, 'stage1')     # calwebb_detector1 pipeline outputs will go here

# Define output subdirectories for new emicorr version
det1_dir_new = os.path.join(obs_dir, 'stage1new')       

# Create desired output directories, if needed
os.makedirs(det1_dir, exist_ok=True)
os.makedirs(det1_dir_new, exist_ok=True)

<hr style="border:1px solid gray"> </hr>

5.<font color='white'>-</font>Detector1 Pipeline<a class="anchor" id="det1"></a>
------------------
In this section, the uncalibrated data are processed through the Detector1
pipeline to create Stage 1 data products, which include 2D countrate
images that have been averaged over all integrations (`*_rate.fits` files) and 3D cubes containing fitted ramp slopes for each integration (`*_rateints.fits` files).  For TSOs, the integrations must be calibrated separately in the subsequent stages, so the `*_rateints.fits` files are the relevant outputs. The Stage 1 data products have units of DN/s.<br>
    
Unlike in the case of MIRI LRS slit mode observations, the `firstframe` and `rscd` steps are skipped by default of TSOs.<br>

See https://jwst-docs.stsci.edu/jwst-science-calibration-pipeline/stages-of-jwst-data-processing/calwebb_detector1 for a detailed overview of the various pipeline steps that comprise Detector1.

### Processing Using Standard EMICORR Algorithm

<div class="alert alert-block alert-warning">
To override certain steps and reference files, use the examples provided below.<br>
E.g., turn on detection of cosmic ray showers.
</div>

Set up a dictionary to define how the Detector1 pipeline should be configured.

In [ ]:
time_det1 = time.perf_counter()

In [ ]:
# Boilerplate dictionary setup
det1dict = {}
det1dict['group_scale'], det1dict['dq_init'], det1dict['emicorr'], det1dict['saturation'], det1dict['ipc'] = {}, {}, {}, {}, {}
det1dict['firstframe'], det1dict['lastframe'], det1dict['reset'], det1dict['linearity'], det1dict['rscd'] = {}, {}, {}, {}, {}
det1dict['dark_current'], det1dict['refpix'], det1dict['charge_migration'], det1dict['jump'], det1dict['ramp_fit'] = {}, {}, {}, {}, {}
det1dict['gain_scale'] = {}

# Turn on detection of cosmic ray showers if desired (off by default)
det1dict['jump']['find_showers'] = True

# Adjust the flagging threshold for cosmic rays (default is 3.0)
det1dict['jump']['rejection_threshold'] = 5.0

Select for only the science data from the target observation, excluding target acquisition and/or pointing verification exposures. For the demo example, there should be only one file, corresponding to a contiguous segment of time-resolved integrations, but for longer TSOs, the full series of exposures may be split into several segments due to file size constraints.

In [ ]:
# Grab all downloaded uncal files
uncal_files = sorted(glob.glob(os.path.join(uncal_dir, '*_uncal.fits')))

# Only choose science exposures, which have the exposure type setting 'MIR_LRS-FIXEDSLIT'
input_files = np.array([fi for fi in uncal_files if fits.getheader(fi, 'PRIMARY')['EXP_TYPE'] == 'MIR_LRS-SLITLESS'])

print('Found ' + str(len(input_files)) + ' science uncal files')

Run the Detector1 pipeline on the selected uncalibrated data using the call method. This process may take several minutes per file.

In [ ]:
# Run the pipeline on the selected input files one by one with the custom parameter dictionary 
for file in input_files:
    Detector1Pipeline.call(file, steps=det1dict, save_results=True, output_dir=det1_dir)

### Processing Using Updated EMICORR Algorithm

Repeat above steps, but for the updated `emicorr` algorithm.

In [ ]:
# Set new emicorr method
det1dict['emicorr']['algorithm'] = 'joint'

In [ ]:
# Run the pipeline on the selected input files one by one with the custom parameter dictionary 
for file in input_files:
    Detector1Pipeline.call(file, steps=det1dict, save_results=True, output_dir=det1_dir_new)

In [ ]:
# Print out the time benchmark
time1 = time.perf_counter()
print(f"Runtime so far: {time1 - time0:0.4f} seconds")
print(f"Runtime for Detector1: {time1 - time_det1} seconds")

<hr style="border:1px solid gray"> </hr>

<hr style="border:1px solid gray"> </hr>

6.<font color='white'>-</font>Compare Outputs<a class="anchor" id="compare-outputs"></a>
------------------
Extract fluxes from strips in the `*_rateints.fits` files and generate comparison plots. The fluxes are in units of DN/sec.<br>

In [ ]:
# Read in the first rate files from the output directories
file = sorted(glob.glob(f'{det1_dir}/*rateints.fits'))[0]
imgs = fits.getdata(file, 'SCI')
ngrps = fits.getheader(file, 'PRIMARY')['NGROUPS']
nints = fits.getheader(file, 'PRIMARY')['NINTS']
file_new = sorted(glob.glob(f'{det1_dir_new}/*rateints.fits'))[0]
imgs_new = fits.getdata(file_new, 'SCI')

print("Number of integrations = {}".format(nints))
print("Number of groups per integration = {}".format(ngrps))

# Make normal plots
%matplotlib inline
# Interactive plots
#%matplotlib notebook

# For first (up to) 10 integrations, extract flux across the subarray and plot
for i in range(min(nints,10)):
    flux = np.nanmedian(imgs[i][2:, 12:], axis=1)
    flux_new = np.nanmedian(imgs_new[i][2:, 12:], axis=1)

    rc('axes', linewidth=2)
    fig, ax = plt.subplots(1, 1, figsize=(10, 5), dpi=150)
    ax.plot(np.arange(len(flux)), flux, color='blue', ls=':', lw=2, label='Standard EMICORR')
    ax.plot(np.arange(len(flux_new)), flux_new, color='blue', ls='-', lw=2, label='New EMICORR')
    plt.xlabel('Row number [px]')
    plt.ylabel('Flux (DN/sec)')
    plt.grid()
    plt.legend(loc='best')
    plt.tight_layout()
    plt.savefig(os.path.join(obs_dir, f'emicorr_test_int{i+1}.png'))
    plt.close()

<hr style="border:1px solid gray"> </hr>

In [ ]:
if do_viz:
    # Get x1dints files
    x1d_file = os.path.join(tso3_dir, 'Stage3_x1dints.fits')

    # Choose arbitrary wavelength elements to extract
    wave_idxs = [100, 200, 350]
    colors = ['r','g','b']

    # Read in file
    hdul = fits.open(x1d_file)
    int_times = hdul[1].data['int_mid_MJD_UTC']
    spec_tables = hdul[2:-1]
    wave = spec_tables[0].data['WAVELENGTH']

    # Plot light curves
    rc('axes', linewidth=2)
    fig, ax = plt.subplots(1, 1, figsize=(10, 5), dpi=150)
    for ii in range(len(wave_idxs)):
        lc = np.array([spec_tables[k].data['FLUX'][wave_idxs[ii]] for k in range(len(int_times))])
        ax.plot(int_times, lc, colors[ii]+'-', lw=2, label="{:.3f} um".format(wave[wave_idxs[ii]]))
    plt.xlabel('MJD_UTC (d)')
    plt.ylabel('Flux (Jy)')
    plt.grid()
    plt.legend(loc='best')
    plt.tight_layout()
    plt.savefig(os.path.join(tso3_dir, 'lrs_slitless_example_speclcs.png'))
    hdul.close()


<hr style="border:1px solid gray"> </hr>

<img style="float: center;" src="https://github.com/spacetelescope/jwst-pipeline-notebooks/raw/main/_static/stsci_footer.png" alt="stsci_logo" width="200px"/> 